This notebook has been streamlined a bit. In the top section, we do the following:
- Create our 3 agents for 2023 - 2025.
- This requires creating instruction contexts for each.
- And creating tools for each. So we import our data and create some simple tools.
- Then we turn each of these agents into tools themselves.
- Then we create the supervisor agent and give it access to the subagents.
- Now we're up & running.

In [67]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [68]:
load_dotenv(override=True)

True

In [ ]:

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("jim@masinyusane.org")
    to_email = To("jim@masinyusane.org")
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

202


This is some background info for us to dynamically plug into each of the agent contexts

In [70]:
zz_background = """# PROGRAMME OVERVIEW
- Programme: **Zazi iZandi** (South Africa)
- Intervention: Teaching small groups of **7 children** their letter sounds in a **frequency‑based sequence**.
- Groups are **level‑based**: each group may be working on different letters at any given time.
- Teacher Assistants (TAs) use an official **Letter Tracker** ordered by letter frequency."""

In [71]:
instructions_2023 = f"""You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}. 
In 2023, Zazi iZandi was piloted in 12 schools. The pilot ran for 3 months, from August to October. 52 youth were hired to work with 1897 children. 

# RESULTS
 
The Grade 1 children improved their Early Grade Reading Assessment (EGRA) scores from 24 to 47.
The Grade 1 children improved the number of letters they knew from 13 to 21. 
The percent of Grade 1 children that reached the target Reading Benchmark increased to 74%.
The Grade R children improved their EGRA scores from 5 to 26.
The Grade R children improved the number of letters they knew from 3 to 12. 

#TOOLS
If you need to know the number of children on the programme, you can use the get_2023_number_of_children function."""


In [72]:
instructions_2024 = f"""You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}. 
In 2024, Zazi iZandi was piloted in 16 schools. 82 youth were hired to work with 3490 children. 

# RESULTS
 
The Grade 1 children improved their Early Grade Reading Assessment (EGRA) scores from 14 to 38.
The percent of Grade 1 children that reached the target Reading Benchmark increased from 13%to 53%.
The Grade R children improved their EGRA scores from 1 to 25.

#TOOLS
If you need to know the number of children on the programme, you can use the get_2023_number_of_children function."""

In [73]:
instructions_2025 = f"""You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2025. {zz_background}. 
Your information is updated through June, so we're only halfway through the year. 
In 2025, Zazi iZandi was piloted in 16 schools. 66 youth were hired to work with 2352 children. A further 20 youth were hired to pilot the programme in 15 Early Childhood Development Centers (ECDs) with 4-6 year old children.

# RESULTS
 
The Grade 1 children improved their Early Grade Reading Assessment (EGRA) scores from 12 to 22.
The percent of Grade 1 children that reached the target Reading Benchmark increased from 6%to 17%.
The Grade R children improved their EGRA scores from 2 to 10.
The Early Childhood Development (ECD) children improved their EGRA scores from 1 to 6.5.

#TOOLS
If you need to know the number of children on the programme, you can use the get_2025_number_of_children function."""

Okay, now let's pull in our data

In [74]:
import os
import sys
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import streamlit as st

# We need to set root directory so we can find the zz_data_process_23.py file. I should obviously move this into a better named directory, will do so in the future.
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if root_dir not in sys.path:
    sys.path.append(root_dir)

#2023 Data
from zz_data_process_23 import process_zz_data_23
from data_loader import load_zazi_izandi_2023

def import_2023_results():
    # Import dataframes
    endline_df, sessions_df = load_zazi_izandi_2023()
    endline = process_zz_data_23(endline_df, sessions_df)
    return endline

#2024 Data
from zz_data_processing import process_zz_data_midline, process_zz_data_endline, grade1_df, gradeR_df
from data_loader import load_zazi_izandi_2024

def import_2024_results():
    baseline_df, midline_df, sessions_df, baseline2_df, endline_df, endline2_df = load_zazi_izandi_2024()
    
    # Create deep copies to ensure data independence between tabs
    baseline_df = baseline_df.copy()
    midline_df = midline_df.copy()
    sessions_df = sessions_df.copy()
    endline_df = endline_df.copy()

    midline, baseline = process_zz_data_midline(baseline_df, midline_df, sessions_df)
    endline = process_zz_data_endline(endline_df)
    grade1 = grade1_df(endline)
    gradeR = gradeR_df(endline)
    
    return endline

root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if root_dir not in sys.path:
    sys.path.append(root_dir)
    
from process_survey_cto_updated import process_egra_data
from data_loader import load_zazi_izandi_2025

def import_2025_results():
    # Load data
    df_full, df_ecd = load_zazi_izandi_2025()
    df_full['submission_date'] = pd.to_datetime(df_full['date'])

    # Create initial and midline datasets for comparison charts
    initial_df = df_full[df_full['submission_date'] < pd.Timestamp('2025-04-15')]
    midline_df = df_full[df_full['submission_date'] >= pd.Timestamp('2025-04-15')]
    
    return midline_df

Okay, let's create a super simple tools for each year.

In [75]:
@function_tool
def get_2023_number_of_children():
    """
    Get the number of children on the programme in 2023
    """
    endline_2023 = import_2023_results()
    number_of_children = len(endline_2023)
    return number_of_children

@function_tool
def get_2024_number_of_children():
    """
    Get the number of children on the programme in 2024
    """
    endline_2024 = import_2024_results()
    number_of_children = len(endline_2024)
    return number_of_children

@function_tool
def get_2025_number_of_children():
    """
    Get the number of children on the programme in 2024
    """
    midline_2025 = import_2025_results()
    number_of_children = len(midline_2025)
    return number_of_children

Let's create our 3 agents.

In [76]:
zazi_2023_agent = Agent(
        name="Zazi 2023 Agent",
        instructions=instructions_2023,
        model="gpt-4o-mini",
        tools=[get_2023_number_of_children]
)

zazi_2024_agent = Agent(
        name="Zazi 2024 Agent",
        instructions=instructions_2024,
        model="gpt-4o-mini",
        tools=[get_2024_number_of_children]
)

zazi_2025_agent = Agent(
        name="Zazi 2025 Agent",
        instructions=instructions_2025,
        model="gpt-4o-mini",
        tools=[get_2025_number_of_children]
)

Let's make the agents tools.

In [77]:
tool_2023 = zazi_2023_agent.as_tool(tool_name="2023_researcher", tool_description="Provides information, data, and statistics about the Zazi iZandi 2023 programme.")
tool_2024 = zazi_2024_agent.as_tool(tool_name="2024_researcher", tool_description="Provides information, data, and statistics about the Zazi iZandi 2024 programme.")
tool_2025 = zazi_2025_agent.as_tool(tool_name="2025_researcher", tool_description="Provides information, data, and statistics about the Zazi iZandi 2025 programme.")


Let's create our supervisor agent.

In [78]:
instructions_supervisor = """You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme. 
{zz_background}.

If the user asks for information about the 2023 programme, you can use the zazi_2023_agent.
If the user asks for information about the 2024 programme, you can use the zazi_2024_agent.

If no year is specified, assume the user in inquiring about 2024.
"""



In [79]:
zazi_supervisor = Agent(
        name="Zazi Supervisor",
        instructions=instructions_supervisor,
        model="gpt-4o",
        tools=[tool_2023, tool_2024, tool_2025]
)

Let's quickly test the agentic system.

In [80]:
question = "How did the children perform in 2024?"

with trace("Zazi iZandi Supervisor Agent"):
    result = await Runner.run(zazi_supervisor, question)

print(result.final_output)

In 2024, the Zazi iZandi literacy programme showed significant improvements for children:

### Grade 1 Performance:
- **EGRA Score Improvement**: Increased from **14** to **38**.
- **Percentage of Children Reaching Reading Benchmark**: Rose from **13%** to **53%**.

### Grade R Performance:
- **EGRA Score Improvement**: Increased from **1** to **25**.

These results indicate strong progress in literacy skills for both grades. If you need more detailed analysis, feel free to ask!


In [81]:
from IPython.display import Markdown, display
display(Markdown(result.final_output))

In 2024, the Zazi iZandi literacy programme showed significant improvements for children:

### Grade 1 Performance:
- **EGRA Score Improvement**: Increased from **14** to **38**.
- **Percentage of Children Reaching Reading Benchmark**: Rose from **13%** to **53%**.

### Grade R Performance:
- **EGRA Score Improvement**: Increased from **1** to **25**.

These results indicate strong progress in literacy skills for both grades. If you need more detailed analysis, feel free to ask!

# New Lesson 4 Content

Let's have our supervisor return with some context for the user. We could create a context agent, but it's probably best to add into the initial instructions. I'm updating the instructions below.

In [82]:
instructions_supervisor = """
You are a helpful, insightful data analyst supporting the user in understanding the performance of the Zazi iZandi literacy programme. Your goal is not just to present raw data, but to interpret it, highlight what is significant, and help the user understand why the results matter.

{zz_background}

Key responsibilities:
1. When the user requests performance data, provide the requested numbers clearly.
2. Where possible, **benchmark these results** against national and provincial norms using the research summaries below.
3. Highlight meaningful comparisons or gains (e.g. "In 2024, 53% of our Grade 1 learners were reading at grade level, compared to only 27% nationally").
4. If results are concerning or below benchmarks, gently and constructively identify these areas for improvement.
5. Offer **plain-language** summaries alongside numbers, to help users (who may not be statisticians) understand the impact.

Benchmarks and context you may refer to:
- **By end of Grade 1**, fewer than 50% of South African learners in no-fee schools know all letters.
- **Only 27% of Eastern Cape Grade 1 learners** reach 40 letters-per-minute (lpm) by year end.
- In Nguni languages, only **7–32% of learners** hit the 40 letters per minute benchmark by end of Grade 1/start of Grade 2.
- **Median fluency in Grade 2** nationally is 11 correct words per minute (benchmark = 30+).
- Only **8–15% of Eastern Cape learners** meet the Grade 4 benchmark of 90 cwpm.
- Pre-pandemic, **more than 55%** of Nguni/Sesotho-Setswana Grade 1 learners couldn’t read a single word from a grade-level text.
- Girls outperform boys significantly in reading as grades progress.

You can use:
- `zazi_2023_agent` for 2023 programme information
- `zazi_2024_agent` for 2024 programme information
- If no year is specified, assume the user means 2024.

Always aim to provide both **data and narrative** so users can make informed decisions and communicate the programme’s impact effectively.
"""


In [83]:
zazi_supervisor = Agent(
        name="Zazi Supervisor",
        instructions=instructions_supervisor,
        model="gpt-4o",
        tools=[tool_2023, tool_2024, tool_2025]
)

Let's retest the agent w/ the updated context.

In [84]:
question = "How did the children perform in 2024?"

with trace("Zazi iZandi Supervisor Agent"):
    result = await Runner.run(zazi_supervisor, question)

print(result.final_output)

In 2024, the Zazi iZandi literacy programme achieved notable success. Here’s a summary of the performance data:

### Programme Performance

#### Grade 1 Children:
- **EGRA Score Improvement**: Increased from **14** to **38**.
- **Percentage Reaching Reading Benchmark**: Climbed from **13%** to **53%**.

**Interpretation:**
- This means that over half of the children reached the reading benchmark, which is a significant achievement compared to the national figure where only 27% of Eastern Cape Grade 1 learners hit the 40 letters per minute benchmark.
- The improvement in EGRA scores indicates substantial progress in literacy skills.

#### Grade R Children:
- **EGRA Score Improvement**: Jumped from **1** to **25**.

**Interpretation:**
- This considerable improvement in the EGRA score shows a strong foundational boost even before entering Grade 1.

### Programme Reach

In 2024, the programme:
- Was piloted in 16 schools.
- Employed 82 youth.
- Served 3,490 children.

### Narrative Summar

# Next Steps
1. Add more utility to the 2023 and 2024 tools.
2. Create a 2025 agent (w/ corresponding tools).
3. Add to the supervisor prompt so that it includes some context.
4. Add in some RAG so that the agent can reference some Q&A.


# Things we won't cover, but we'll eventually use to enhance our bot.
5. Add in a guardrail so that no personal information is returned to the user.
6. Add in some structured outputs.
7. Add in a tool for the agent to email us if someone wants to connect. It should get their personal info.
8. Add in a tool to send us a push notification and/or email if the LLM cannot answer a question.